# ResNet Implementation with TensorFlow

## Download CIFAR10

In [1]:
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data

In [2]:
(trainX, trainY), (testX, testY) = load_data()

In [3]:
print(trainX.shape)
print(testX.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [4]:
print(trainY.shape)
print(testY.shape)

(50000, 1)
(10000, 1)


In [31]:
train_num = int(len(trainX)*0.9)
valid_num = len(trainX) - train_num

In [33]:
validX = trainX[-valid_num:]
validY = trainY[-valid_num:]

trainX = trainX[:train_num]
trainY = trainY[:train_num]

## Construct Model

In [6]:
input_shape = trainX.shape[1:]

In [7]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, MaxPool2D, Dense
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import add
from tensorflow.keras.activations import relu
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [8]:
# Basic Block

def make_residual(num_filters, stride):
    def f(x):
        x = Conv2D(filters=num_filters,
                   kernel_size=(3,3),
                   strides=stride, padding='same',
                   activation=None,
                   use_bias=False, kernel_initializer='he_uniform')(x)
        x = BatchNormalization()(x)
        x = relu(x)
        x = Conv2D(filters=num_filters,
                   kernel_size=(3,3),
                   strides=1, padding='same',
                   use_bias=False, kernel_initializer='he_uniform')(x)
        x = BatchNormalization()(x)
        
        return x
    
    return f

In [9]:
channels=[16,16,32,64,128]

In [10]:
input_shape

(32, 32, 3)

In [17]:
input_tensor=Input(input_shape)

x = Conv2D(filters=channels[0], kernel_size=(7,7), padding='same', activation=None, use_bias=False, kernel_initializer='he_uniform')(input_tensor)
x = BatchNormalization()(x)
x = relu(x)

# Block 1
x = add([make_residual(channels[0], 1)(x), x])

# Block 2
y = make_residual(channels[1], 2)(x)
z = Conv2D(filters=channels[1], kernel_size=(1,1), strides=2, padding='same', activation=None)(x)
x = add([y,z])

# Block 3
y = make_residual(channels[2], 1)(x)
z = Conv2D(filters=channels[2], kernel_size=(1,1), strides=1, padding='same', activation=None)(x)
x = add([y,z])

# Block 4
y = make_residual(channels[3], 2)(x)
z = Conv2D(filters=channels[3], kernel_size=(1,1), strides=2, padding='same', activation=None)(x)
x = add([y,z])

# Block 5
y = make_residual(channels[4], 2)(x)
z = Conv2D(filters=channels[4], kernel_size=(1,1), strides=2, padding='same', activation=None)(x)
x = add([y,z])

# Block 6
x = add([make_residual(channels[4], 1)(x), x])

x = GlobalAveragePooling2D()(x)

x = Dense(channels[4]//4, activation=None, use_bias=False, kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = relu(x)
x = Dense(10, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=x)

optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 32, 32, 16)   2352        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 32, 32, 16)   64          conv2d_17[0][0]                  
__________________________________________________________________________________________________
tf.nn.relu_8 (TFOpLambda)       (None, 32, 32, 16)   0           batch_normalization_14[0][0]     
____________________________________________________________________________________________

In [29]:
from tensorflow.keras.callbacks import LearningRateScheduler

lr = LearningRateScheduler(lambda epoch: 0.01*(0.95 ** epoch), verbose=1)

In [37]:
history=model.fit(trainX, trainY, batch_size=128, epochs=10, validation_data=(validX, validY), callbacks=[lr])

Epoch 1/10

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
 15/282 [>.............................] - ETA: 1:23 - loss: 2.2882 - accuracy: 0.1141

KeyboardInterrupt: 

In [34]:
test_loss, test_acc = model.evaluate(testX, testY, verbose=1)

313/313 [==============================] - 4s 13ms/step - loss: 2.2887 - accuracy: 0.1069


## Plot Training Loss and Validation loss 

In [36]:
plt.figure()
plt.plot(np.arange(10), history.history['loss'])
plt.plot(np.arange(10), history.history['val_loss'])
plt.show()

NameError: name 'history' is not defined

<Figure size 640x480 with 0 Axes>